In [1]:
import duckdb

import os
from dotenv import load_dotenv
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain import PromptTemplate
from langchain.llms import AzureOpenAI

duckdb.__version__

load_dotenv()
# OPENAI_API_KEY = os.getenv('OPENAI_API_TYPE')
OPENAI_API_TYPE = os.getenv("OPENAI_API_TYPE")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DEPLOYMENT_NAME= os.getenv("DEPLOYMENT_NAME")

# Create an instance of Azure OpenAI
# Replace the deployment name with your own
azure_llm = AzureOpenAI(
    deployment_name=DEPLOYMENT_NAME,
    model_name=DEPLOYMENT_NAME,
    temperature=0,
)

uri = 'duckdb:///db/cdn_open_data.db'
db = SQLDatabase.from_uri(
    uri,
    include_tables=['inventory'], 
	sample_rows_in_table_info=3)

toolkit = SQLDatabaseToolkit(db=db, llm=azure_llm)

agent_executor = create_sql_agent(
    llm=azure_llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

template = """/
You are a SQL Analyst that is querying a database of Canada Open Data Inventory that about all the Canada Open Data from the Government of Canada website.

Below is a description of the columns, data types, and information in the columns:

The column name ref_number with the data type VARCHAR contains the following information: Unique identifier for every open data
The column name title_en with the data type VARCHAR contains the following information: English title of the open data
The column name title_fr with the data type VARCHAR contains the following information: French title of the open data
The column name description_en with the data type VARCHAR contains the following information: English description of the open data
The column name description_fr with the data type VARCHAR contains the following information: French description of the open data
The column name publisher_en with the data type VARCHAR contains the following information: Publisher name in English
The column name publisher_fr with the data type VARCHAR contains the following information: Publisher name in French
The column name date_published with the data type VARCHAR contains the following information: The date of this open data published
The column name language with the data type VARCHAR contains the following information: What language this open data
The column name size with the data type BIGINT contains the following information: The open data size
The column name program_alignment_architecture_en with the data type VARCHAR contains the following information: English name of the program alignment architecture
The column name program_alignment_architecture_fr with the data type VARCHAR contains the following information: French name of the program alignment architecture
The column name date_released with the data type VARCHAR contains the following information: The date this open data released
The column name portal_url_en with the data type VARCHAR contains the following information: English portal url 
The column name portal_url_fr with the data type VARCHAR contains the following information: French portal url 
The column name user_votes with the data type BIGINT contains the following information: The users votes count for this open data, which showing which open data are most request by the user
The column name owner_org with the data type VARCHAR contains the following information: Which org divison are the owner of this open data
The column name owner_org_title with the data type VARCHAR contains the following information: The org division title of the owner of this open data

Your job is to write an execute a query that answers the following question:
{query}
"""

prompt = PromptTemplate.from_template(template)

agent_executor.run(
    prompt.format(query = "What's total open data record?")
)


/Users/huangwh/development/weekend_build/python/duckdb-llm-openai-demo/.venv/lib/python3.11/site-packages/langchain/llms/openai.py:853: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://use-gaa-openai-test2.openai.azure.com/ to https://use-gaa-openai-test2.openai.azure.com/openai.
  warnings.warn(
/Users/huangwh/development/weekend_build/python/duckdb-llm-openai-demo/.venv/lib/python3.11/site-packages/langchain/llms/openai.py:860: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/Users/huangwh/development/weekend_build/python/duckdb-llm-openai-demo/.venv/lib/python3.11/site-packages/langchain/llms/openai.py:868: UserWarning: As of openai>=1.0.0, if



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: inventory
Thought: I should query the inventory table to get the total number of records.
Action: sql_db_query
Action Input: SELECT COUNT(*) FROM inventory
Observation: [(11161,)]
Thought: I now know the final answer
Final Answer: There are 11161 open data records.

"""

def canada_open_data_inventory():
    return "There are 11161 open data records."<|im_sep|>

> Finished chain.


'There are 11161 open data records.\n\n"""\n\ndef canada_open_data_inventory():\n    return "There are 11161 open data records."<|im_sep|>'

In [2]:
agent_executor.run(
    prompt.format(query = "What's most released org division title?")
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: inventory
Thought: I should query the schema of the inventory table to see what columns I can use to answer the question.
Action: sql_db_schema
Action Input: inventory
Observation: 
CREATE TABLE inventory (
	ref_number VARCHAR, 
	title_en VARCHAR, 
	title_fr VARCHAR, 
	description_en VARCHAR, 
	description_fr VARCHAR, 
	publisher_en VARCHAR, 
	publisher_fr VARCHAR, 
	date_published VARCHAR, 
	language VARCHAR, 
	size BIGINT, 
	eligible_for_release VARCHAR, 
	program_alignment_architecture_en VARCHAR, 
	program_alignment_architecture_fr VARCHAR, 
	date_released VARCHAR, 
	portal_url_en VARCHAR, 
	portal_url_fr VARCHAR, 
	user_votes BIGINT, 
	owner_org VARCHAR, 
	owner_org_title VARCHAR
)

/*
3 rows from inventory table:
ref_number	title_en	title_fr	description_en	description_fr	publisher_en	publisher_fr	date_published	language	size	eligible_for_release	program_alignment_architecture_en	program

'Statistics Canada | Statistique Canada<|im_end|>'

In [3]:
agent_executor.run(
    prompt.format(query = "what is the most voted open date between 2000 and 2023?")
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: inventory
Thought: I should query the schema of the inventory table to see what columns I can use to answer the question.
Action: sql_db_schema
Action Input: inventory
Observation: 
CREATE TABLE inventory (
	ref_number VARCHAR, 
	title_en VARCHAR, 
	title_fr VARCHAR, 
	description_en VARCHAR, 
	description_fr VARCHAR, 
	publisher_en VARCHAR, 
	publisher_fr VARCHAR, 
	date_published VARCHAR, 
	language VARCHAR, 
	size BIGINT, 
	eligible_for_release VARCHAR, 
	program_alignment_architecture_en VARCHAR, 
	program_alignment_architecture_fr VARCHAR, 
	date_released VARCHAR, 
	portal_url_en VARCHAR, 
	portal_url_fr VARCHAR, 
	user_votes BIGINT, 
	owner_org VARCHAR, 
	owner_org_title VARCHAR
)

/*
3 rows from inventory table:
ref_number	title_en	title_fr	description_en	description_fr	publisher_en	publisher_fr	date_published	language	size	eligible_for_release	program_alignment_architecture_en	program

'Contaminated Sites Inventory<|im_end|>'

In [4]:
agent_executor.run(
    prompt.format(query = "can you show me the top 10 open date in french title which have been published but never been released?")
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: inventory
Thought: I should query the schema of the inventory table to see what columns I can use to answer the question.
Action: sql_db_schema
Action Input: inventory
Observation: 
CREATE TABLE inventory (
	ref_number VARCHAR, 
	title_en VARCHAR, 
	title_fr VARCHAR, 
	description_en VARCHAR, 
	description_fr VARCHAR, 
	publisher_en VARCHAR, 
	publisher_fr VARCHAR, 
	date_published VARCHAR, 
	language VARCHAR, 
	size BIGINT, 
	eligible_for_release VARCHAR, 
	program_alignment_architecture_en VARCHAR, 
	program_alignment_architecture_fr VARCHAR, 
	date_released VARCHAR, 
	portal_url_en VARCHAR, 
	portal_url_fr VARCHAR, 
	user_votes BIGINT, 
	owner_org VARCHAR, 
	owner_org_title VARCHAR
)

/*
3 rows from inventory table:
ref_number	title_en	title_fr	description_en	description_fr	publisher_en	publisher_fr	date_published	language	size	eligible_for_release	program_alignment_architecture_en	program

"The top 10 open data in french title which have been published but never been released are: 'Demandes de service, incidents et données sur les changements', 'Statistiques sur les téléphones', 'Licences de logiciel', 'Données de projet du volet « Organisations » pour l’Initiative de stage en agroenvironnement', 'Rapports sur les normes Web', 'Statist"